In [4]:
!wget "https://github.com/romenlaw/NaiveNeuralNetwork/blob/main/names.txt?raw=True" -O names.txt

--2024-08-28 01:32:22--  https://github.com/romenlaw/NaiveNeuralNetwork/blob/main/names.txt?raw=True
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/romenlaw/NaiveNeuralNetwork/raw/main/names.txt [following]
--2024-08-28 01:32:22--  https://github.com/romenlaw/NaiveNeuralNetwork/raw/main/names.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/romenlaw/NaiveNeuralNetwork/main/names.txt [following]
--2024-08-28 01:32:22--  https://raw.githubusercontent.com/romenlaw/NaiveNeuralNetwork/main/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting

In [5]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
words = open('names.txt', 'r').read().splitlines()
len(words), words[:8]

(32033,
 ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia'])

In [15]:
vocab = sorted(list(set(''.join(words))))
vocab.insert(0, '.')
vocab_size = len(vocab)
itos={i:s for i, s in zip(range(vocab_size), vocab)}
stoi={s:i for i, s in zip(range(vocab_size), vocab)}

In [17]:
# build datasets: 80% training, 10% validation, 10% testing
import random
random.seed(42)
random.shuffle(words) # shuffle is in-place
n1 = int(len(words)*.8)
n2 = int(len(words)*.9)

block_size=3
X, Y = [], []


{0: '.',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}